In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
dataset = pd.read_csv("./dataset.csv")

# Group by 'test', 'frame', 'label', and 'type'
grouped_data = dataset.groupby(['test', 'frame', 'label', 'type'])

# Lists to store data based on type and label
fetus_baseline = []
fetus_opcl = []
fetus_yawn = []
mother_baseline = []
mother_opcl = []
mother_yawn = []

# Process grouped data
for name, group in grouped_data:
    group = group.drop(group.columns[0], axis=1)  # Drop the first column
    group = group.drop(["leftLip_x", "leftLip_y", "rightLip_x", "rightLip_y", 
                        "topMidInner_x", "topMidInner_y", "bottomMidInner_x", 
                        "bottomMidInner_y", "nose_x", "nose_y"], axis=1)
    
    # Get values from 'top_bottom_distance'
    top_bottom_distance_values = group['top_bottom_distance'].values
    
    # Assign to appropriate lists based on label and type
    label = name[2]  # 'label' from groupby
    type_ = name[3]  # 'type' from groupby
    
    if type_ == 'fetus':
        if label == 'baseline':
            fetus_baseline.append(top_bottom_distance_values)
        elif label == 'opcl':
            fetus_opcl.append(top_bottom_distance_values)
        elif label == 'yawn':
            fetus_yawn.append(top_bottom_distance_values)
    elif type_ == 'mother':
        if label == 'baseline':
            mother_baseline.append(top_bottom_distance_values)
        elif label == 'opcl':
            mother_opcl.append(top_bottom_distance_values)
        elif label == 'yawn':
            mother_yawn.append(top_bottom_distance_values)

# Function to pad, sample, and smooth each array
def process_series(series_list, window_size=3):
    max_length = max(len(series) for series in series_list)
    padded_series = np.array([np.pad(series, (0, max_length - len(series)), mode='constant') for series in series_list])

    # Sampled series
    sampled_series = []
    sample_n = 1
    for i in range(0, max_length, sample_n):
        segment_values = padded_series[:, i:i + sample_n][padded_series[:, i:i + sample_n] != 0]
        if len(segment_values) > 0:
            sampled_point = np.random.choice(segment_values.flatten(), size=1)
            sampled_series.append(sampled_point[0])
    
    sampled_series = np.array(sampled_series)

    # Apply moving average for smoothing
    smooth_sampled_series = np.convolve(sampled_series, np.ones(window_size) / window_size, mode='valid')
    
    return smooth_sampled_series

# Process all series
smooth_fetus_baseline = process_series(fetus_baseline)
smooth_fetus_opcl = process_series(fetus_opcl)
smooth_fetus_yawn = process_series(fetus_yawn)
smooth_mother_baseline = process_series(mother_baseline)
smooth_mother_opcl = process_series(mother_opcl)
smooth_mother_yawn = process_series(mother_yawn)

# Function to plot and save images
def plot_and_save(series_dict, title, filename):
    plt.figure(figsize=(19.20, 10.80))
    for label, series in series_dict.items():
        plt.plot(series, marker='x', label=label, linestyle='--')
    
    plt.title(title)
    plt.xlabel('Time Points')
    plt.ylabel('Values')
    plt.legend()
    plt.grid()
    
    # Save the plot
    plt.savefig(filename)
    plt.close()  # Close the figure to save memory

# Create plots for fetus and mother
fetus_series_dict = {
    'Smoothed Fetus Baseline': smooth_fetus_baseline,
    'Smoothed Fetus OPCL': smooth_fetus_opcl,
    'Smoothed Fetus Yawn': smooth_fetus_yawn,
}

mother_series_dict = {
    'Smoothed Mother Baseline': smooth_mother_baseline,
    'Smoothed Mother OPCL': smooth_mother_opcl,
    'Smoothed Mother Yawn': smooth_mother_yawn,
}

# Plot and save images
plot_and_save(fetus_series_dict, 'Smoothed Series for Fetus', 'fetus_series.png')
plot_and_save(mother_series_dict, 'Smoothed Series for Mother', 'mother_series.png')

# Combine the plots into a single figure
plt.figure(figsize=(19.20, 10.80))

# Plot all series together
for label, series in fetus_series_dict.items():
    plt.plot(series, marker='x', label=label, linestyle='--')
for label, series in mother_series_dict.items():
    plt.plot(series, marker='o', label=label, linestyle='--')

plt.title('Combined Smoothed Series for Fetus and Mother')
plt.xlabel('Time Points')
plt.ylabel('Values')
plt.legend()
plt.grid()

# Save the combined plot
plt.savefig('combined_fetus_mother_series.png')
plt.close()
